## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [1]:
# Import necessary libraries
from dotenv import load_dotenv  # For loading environment variables from a .env file
from openai import OpenAI  # For interacting with OpenAI services
import json  # For handling JSON data
import os  # For interacting with the operating system
import requests  # For making HTTP requests
from pypdf import PdfReader  # For reading PDF files
import gradio as gr  # For querying JSON data using JMESPath syntax
from string import Template  # For creating string templates with placeholders

In [2]:
# Load environment variables from a .env file
load_dotenv(override = True)  # This loads the variables defined in the .env file into the environment

# Initialize the OpenAI client with the retrieved credentials
openai = OpenAI()

In [3]:
# Retrieve Pushover API credentials from environment variables
pushover_user = os.getenv("PUSHOVER_USER")  # Get the Pushover user key
pushover_token = os.getenv("PUSHOVER_TOKEN")  # Get the Pushover API token

# Define the Pushover API URL for sending messages
pushover_url = "https://api.pushover.net/1/messages.json"

# Check if the Pushover API user key is available
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")  # Print the first character of the user key
else:
    print("Pushover user not found")  # Notify if the user key is not found

# CHeck if the Pushover API token is available
if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")  # Print the first character of the token
else:
    print("Pushover token not found")  # Notify if the token is not found

Pushover user found and starts with u
Pushover token found and starts with a


In [4]:
# Function to send a notification via Pushover
def push(message):
    # Print the message that will be sent
    print(f"Push : {message}")
    
    # Create the payload for the Pushover API request
    payload = {
        "user" : pushover_user,  # The Pushover user key 
        "token" : pushover_token,  # The Pushover API token
        "message" : message  # The message content to be sent
        }
    
    # Send a POST request to the Pushover API to deliver the message
    requests.post(pushover_url, data = payload)

In [5]:
push("HEY SIDDHARTH HERE")

Push : HEY SIDDHARTH HERE


In [6]:
# Function to record user details and send a notification
def record_user_details(email = "N/A", name = "N/A", mobile_no = "N/A", notes = "N/A"):
    # Create a formatted message for the notification
    notification_message = {
        f"New User Interest Notification:\n"
        f"Name: {name}\n"
        f"Email: {email}\n"
        f"Mobile Bo: {mobile_no}\n"
        f"Notes: {notes}\n"
        f"Please follow up with the user at your earliest convenience."
    }
    
    # Send a notification with the user's details using the push function
    push(notification_message)
    
    # Return a confirmation response indicating that the unknown question has been recorded
    return {"recorded" : "ok"}

In [7]:
# Function to record an unknown question and send a notification
def record_unknown_question(question):
    # Send a notification with the unknown question using the push function
    push(f"Recording {question} asked that I couldn't answer")
    
    # Return a confirmation response indicating that the unknown question has been recorded
    return {"recorded" : "ok"}

In [8]:
# JSON structure for the record_user_details function
record_user_details_json = {
    "name" : "record_user_details",  # The name of the function being called
    
    # Brief overview of the function's purpose
    "description" : "Use this tool to record that a user is interested in being in touch and provided an email address, name, mobile number and notes(optional)", 
    "parameters" : 
        {
            "type" : "object",  # The data type of the parameters (e.g., "object")
            "properties" :  # A collection of key-value pairs representing the function's parameters
                {
                    "email" : 
                        {
                            "type" : "string",  # Expected data type of the parameter
                            "description" : "The email address of this user",  # Explanation of what the parameter represents
                            "default" : "N/A"  # Default value of the parameter
                        }, 
                    "name" : 
                        {
                            "type" : "string",  # Expected data type of the parameter
                            "description" : "The user's name, if they provided it",  # Explanation of what the parameter represents
                            "default" : "N/A"  # Default value of the parameter
                        }, 
                    "mobile_no" : 
                        {
                            "type" : "string",  # Expected data type of the parameter
                            "description" : "The mobile number of this user",  # Explanation of what the parameter represents
                            "default" : "N/A"  # Default value of the parameter
                        }, 
                    "notes" : 
                        {
                            "type" : "string",  # Expected data type of the parameter
                            "description" : "Any additional information about the conversation that's worth recording to give context",  # Explanation of what the parameter represents
                            "default" : "N/A"  # Default value of the parameter
                        }
                }, 
            "required" : ["email"],  # List of parameters that are mandatory for the function to execute
            "additionalProperties" : False  # Indicates whether additional parameters beyond those specified are allowed
        }
}

In [9]:
# JSON structure for the record_unknown_question function
record_unknown_question_json = {
    "name" : "record_unknown_question",  # The name of the function being called
    
    # Brief overview of the function's purpose
    "description" : "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters" : 
        {
            "type" : "object",  # The data type of the parameters (e.g., "object")
            "properties" :  # A collection of key-value pairs representing the function's parameters
                {
                    "question" : 
                        {
                            "type" : "string",  # Expected data type of the parameter
                            "description": "The question that couldn't be answered",  # Explanation of what the parameter represents
                            "default" : "N/A"  # Default value of the parameter
                        }, 
                }, 
            "required" : ["question"],  # List of parameters that are mandatory for the function to execute
            "additionalProperties" : False  # Indicates whether additional parameters beyond those specified are allowed
        }
}

In [10]:
# List of tools to be passed to the LLM
tools = [
    {
        "type" : "function",  # Specifies that this entry is a function tool
        "function" : record_user_details_json  # The JSON structure for the record_user_details function
    }, 
    {
        "type" : "function",  # Specifies that this entry is a function tool
        "function":  record_unknown_question_json  # The JSON structure for the record_unknown_question function
    }
]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address, name, mobile number and notes(optional)',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user',
      'default': 'N/A'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it",
      'default': 'N/A'},
     'mobile_no': {'type': 'string',
      'description': 'The mobile number of this user',
      'default': 'N/A'},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context",
      'default': 'N/A'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question th

In [12]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called : {tool_name}", flush = True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role" : "tool", "content" : json.dumps(result), "tool_call_id" : tool_call.id})
    return results

In [13]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '# Import necessary libraries\nfrom dotenv import load_dotenv  # For loading environment variables from a .env file\nfrom openai import OpenAI  # For interacting with OpenAI services\nimport json  # For handling JSON data\nimport os  # For interacting with the operating system\nimport requests  # For making HTTP requests\nfrom pypdf import PdfReader  # For reading PDF files\nimport gradio as gr  # For querying JSON data using JMESPath syntax\nfrom string import Template  # For creating string templates with placeholders',
  '# Load environment variables from a .env file\nload_dotenv(override = True)  # This loads the variables defined in the .env file into the environment\n\n# Initialize the OpenAI client with t

In [14]:
globals()["record_user_details"]

<function __main__.record_user_details(email='N/A', name='N/A', mobile_no='N/A', notes='N/A')>

In [15]:
globals()["record_user_details"](email = "s.siddharth13@tcs.com", name = "Siddharth Singh", notes = "Sending message from TCS account")

Push : {'New User Interest Notification:\nName: Siddharth Singh\nEmail: s.siddharth13@tcs.com\nMobile Bo: N/A\nNotes: Sending message from TCS account\nPlease follow up with the user at your earliest convenience.'}


{'recorded': 'ok'}

In [16]:
# The globals() function in Python returns a dictionary representing the current global symbol table, 
# allowing dynamic access or modification of global variables within a program.

# Purpose and Use of globals()
# - globals() is typically used to read or modify global variables from anywhere in a program, including from inside functions.
# - This bypasses scoping rules, letting one access or update variables that aren't in a local scope by referencing their 
# names as keys in the dictionary returned by globals().

globals()["record_unknown_question"]("this is a really hard question")

Push : Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [17]:
globals()["record_user_details"](name = "Siddharth Singh", email = "siddharthwolverine@gmail.com", 
                                 notes = "I am working as a Researcher in TCS.")

Push : {'New User Interest Notification:\nName: Siddharth Singh\nEmail: siddharthwolverine@gmail.com\nMobile Bo: N/A\nNotes: I am working as a Researcher in TCS.\nPlease follow up with the user at your earliest convenience.'}


{'recorded': 'ok'}

In [18]:
# Retrieve the function 'record_unknown_question' from the global namespace
func_name = globals().get("record_user_details")

# Check if the function was found (i.e., it is not None)
if func_name is not None:
    # Call the function with the specified question it it exists
    func_name(name = "GIRIMALLA KUREVA", email = "k.girimalla@tcs.com", notes = "Sending Information from Test Environment")
else:
    # Print a message if the function was not found in the global namespace
    print("Function not found.")

Push : {'New User Interest Notification:\nName: GIRIMALLA KUREVA\nEmail: k.girimalla@tcs.com\nMobile Bo: N/A\nNotes: Sending Information from Test Environment\nPlease follow up with the user at your earliest convenience.'}


In [19]:
# Function to handle tool calls made by the LLM
def handle_tool_calls(tool_calls):
    """
    This function processes a list of tool calls generated by the LLM.
    Each tool call is represented as a ChatCompletionMessageToolCall object.
    
    Example of tool_calls:
    [
        ChatCompletionMessageToolCall(
            id="call_mnC1KYpiUrlKYEaRqD9U9",
            function=Function(arguments='{"question":"Can you tell me about Nvidia?"}', name="record_unknown_question"),
            type="function"
        )
    ]
    """
    
    # Initialize an empty list to store the results of each tool call
    results = []
    
    # Iterate over each tool call in the provided list
    for tool_call in tool_calls:
        # Extract the name of the function to be called from the tool call object
        tool_name = tool_call.function.name
        
        # Retrieve the arguments for the function, which are stored as a JSON string
        # Convert the JSON string into a python dictionary for easier access
        arguments = json.loads(tool_call.function.arguments)
        
        # Log the tool name and the arguments being passed for debugging purposes
        print(f"Tool called : {tool_name} || , arguments passed : {arguments}", flush = True)
        
        # Attempt to retrieve the function from the global namespace using its name
        tool = globals().get(tool_name)
        
        # Check if the function was successfully retrieved (i.e., it exists)
        if tool is not None:
            # Validate the arguments against the expected parameters
            expected_params = tool.__code__.co_varnames[:tool.__code__.co_argcount]  # Get expected parameter names
            print(f"Expected parameters for tool '{tool_name}' : {expected_params}")  # Log this for easier understanding
            
            # Check for missing parameters
            missing_params = [param for param in expected_params if param not in arguments]
            if missing_params:
                print(f"Missing parameters for {tool_name} : {missing_params}")
                result = {"error" : f"Missing parameters : {missing_params}"}
            else:
                # Call the function with unpacked arguments and store the result
                result = tool(**arguments)
        else:
            # If the function doesn't exist, initialize the result as an error message
            result = {"error" : f"Function '{tool_name}' not found."}
        
        # Append the result of the function call to the results list
        # Each entry includes the role, the content of the result, and the ID of the tool call
        results.append({
            "role" : "tool",  # Indicates that this entry is a tool result
            "content" : json.dumps(result),  # Convert the result to a JSON string for consistency
            "tool_call_id" : tool_call.id  # Include the unique ID of the tool call for reference
            })

    # Return the compiled list of results from all processed tool calls
    return results

In [20]:
# Read the LinkedIn profile PDF file to extract text
reader = PdfReader("me/personal_linkedIn.pdf")  # Initialize the PDF reader with the specified file
linkedin = ""  # Initialize an empty string to hold the extracted LinkedIn profile tet

# Iterate over each page in the PDF document
for page in reader.pages:
    text = page.extract_text()  # Extract text from the current page
    if text:  # Check if any text was extracted
        linkedin = linkedin + text  # Concatenate the extracted text to the LinkedIn string

# Read the LinkedIn summary from a text file
with open("me/summary.txt", "r", encoding = "utf-8") as f:
    summary = f.read()  # Read the entire content of the summary file into the summary variable

In [21]:
system_prompt = """
You are a professional representative of ${name} on their website, dedicated to answering inquiries about ${name}'s career, background, skills and experience. Your goal is to engage users authentically, reflecting ${name}'s voice and professionalism, as if conversing with a potential client or employer.
You have access to a detailed summary of ${name}'s background and LinkedIn profile, which you should leverage to provide informed and relevant responses.

### Tool Usage:
- **Unknown Questions**: If you encounter a question that you cannot answer based on the provided summary or LinkedIn profile, do not respond to the question at all. For example, if a user asks about ${name}'s favorite movie or unrelated personal interests, simply acknowledge that you cannot provide that information. Use your `record_unknown_question` tool to document the question, including the exact wording. This ensures that all user inquiries are tracked for future reference and can help improve responses in subsequent interactions.
- **User Engagement**: If the user shows interest in further discussions or expresses a desire to connect, actively invite them to provide their name, email. mobile number, and any additional notes they may have. Use your `record_user_details` tool to capture this information. If the user does not provide their name, email, or mobile number, use default values.

## Summary:
${summary}

## LinkedIn Profile:
${linkedin}

With this context, please interact with users, ensuring you remain in character as ${name}.

## Guidelines:
1. **Stay in Character**: Respond as if you are ${name}, maintaining their unique tone and style.
2. **Exude Professionalism**: Your responses should always reflect a professional demeanor suitable for potential clients or employers.
3. **Acknowledge Limitations**: If you lack information on a topic, do not respond to the question. Instead utilize the `record_unknown_question` tool to document the inquiry.
4. **Utilize Context**: Reference the provided summary and LinkedIn profile to enrich your responses.
5. **Foster Engagement**: Encourage users to ask more questions and maintain a lively conversation.
6. **Respect User Inquiries**: Treat all question with respect, providing thoughtful and considerate responses.
7. **Encourage Connection**: Actively invite users to share their name, email, mobile number, and any additional notes for further engagement, ensuring to record it using the appropriate tool. If not provided, use default values for the fields.
8. **Limitations on Knowledge**: Do not use any external knowledge, assumptions, or prior training to answer question. Only respond based on the provided context.
"""

In [22]:
# Function to handle chat interactions with the LLM
def chat(message, history):
    # Construct the messages to be sent to the LLM
    messages = (
        [
            {
                "role" : "system",  # Role of the message sender, indicating this is a system message
                "content" : Template(system_prompt).substitute(
                    name = "Siddharth Singh",  # Substitute the user's name into the system prompt
                    linkedin = linkedin,  # Substitute the LinkedIn profile text into the system prompt
                    summary = summary  # Substitute the summary text into the system prompt
                ),
            }
        ]
        + history  # Include the previous chat history to maintain context
        + [{"role" : "user", "content" : message}]  # Add the current user message to the messages list
    )
    
    done = False  # Initialize a flag to control the loop for processing LLM responses
    while not done:  # Continue processing until the LLM has finished its response
        # Call the LLM with the constructed messages and the tools available for function calls
        response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages, tools = tools)
        print(f"Response from LLM -> {response}")
        
        # Determine the reason for the LLM's response completion
        finish_reason = response.choices[0].finish_reason
        print(f"Finish Reason -> {finish_reason}")  # Log the finish reason for debugging
        
        # If the LLM indicates it wants to call a tool, handle that case
        if finish_reason=="tool_calls":
            # Step 1 - Extract the message from the response, which may contain tool call information
            message = response.choices[0].message
            print(f"Message going inside -> {message}")  # Log the extracted message for debugging
            
            """
            Below is an example of how it may look like -
            Message going inside -> ChatCompletionMessage(content = None, refusal = None, role = "assistant", annotations = [],
            audio = None, function_call = None, tool_calls = [ChatCompletionMessageToolCall(id = "call_mncajkashasklals", 
            function = Function(arguments = '{"question" : "Can you tell me about Nvidia?"}', name = "record_unknown_question"), 
            type = "function")])
            """
            
            # Step 2 - Extract tool_calls details from the 'ChatCompletionMessage'
            tool_calls = message.tool_calls  # Retrieve the list of tool calls requested by the LLM
            print(f"Tool calls -> {tool_calls}")  # Log the tool calls for debugging
            
            """
            Below is an example of how it may look like -
            Message going inside -> [ChatCompletionMessageToolCall(id = "call_mncajkashasklals", 
            function = Function(arguments = '{"question" : "Can you tell me about Nvidia?"}', name = "record_unknown_question"), 
            type = "function")]
            """
            
            # Step 3 - Call the function 'handle_tool_calls' to process the function execution
            results = handle_tool_calls(tool_calls)  # Process the tool calls and obtain results
            print(f"Final Result -> {results}")  # Log the final results for debugging
            
            # Append the LLM's message and the results from tool calls to the message list
            messages.append(message)  # Add the LLM's message to the conversation history
            messages.extend(results)  # Add the results from the tool calls to the conversation history
        else:
            done = True  # Exit the loop if no tool calls are made, indicating the LLM has finished processing
    
    # Return the content of the LLM's response to the user
    return response.choices[0].message.content

In [23]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Response from LLM -> ChatCompletion(id='chatcmpl-CI6e1lIYoHUqPQs77nSACGOXKgm0E', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mE89WGKwi5iO56RsUBMfoi0L', function=Function(arguments='{"question":"Hi, tell me about virat kohli"}', name='record_unknown_question'), type='function')]))], created=1758431481, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=23, prompt_tokens=2054, total_tokens=2077, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=1920)))
Finish Reason -> tool_calls
Message going inside -> ChatCompletionMess

In [24]:
from pydantic import BaseModel, Field

# Define the Evaluation model to assess the quality of LLM responses
class Evaluation(BaseModel):
    is_acceptable : bool = Field(..., description = "Indicates if the response is of acceptable quality.")
    feedback : str = Field(..., description = "Feedback on the response quality.")
    
# Define a base model to allow arbitrary types
class BaseArbitraryModel(BaseModel):
    model_config = {"arbitrary_types_allowed" : True}
    model_config["protected_namespaces"] = ()

In [25]:
# Generate the JSON schema for the Evaluation model
evaluation_json_schema = Evaluation.model_json_schema()

# Display the generated JSON Schema
evaluation_json_schema

{'properties': {'is_acceptable': {'description': 'Indicates if the response is of acceptable quality.',
   'title': 'Is Acceptable',
   'type': 'boolean'},
  'feedback': {'description': 'Feedback on the response quality.',
   'title': 'Feedback',
   'type': 'string'}},
 'required': ['is_acceptable', 'feedback'],
 'title': 'Evaluation',
 'type': 'object'}

In [26]:
# Define the evaluator system prompt for assessing the quality of responses
evaluator_system_prompt = """
You are an evaluator that decides whether a response to a question is acceptable.
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality.
The Agent is playing the role of ${name} and is representing ${name} on their website.
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website.

The Agent has been provided with context on ${name} in the form of their summary and LinkedIn details. Here's the information:
## Summary:
${summary}
## LinkedIn Profile:
${linkedin}

With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback.

## Guidelines:
1. **Stay in Character**: Always respond as if you are ${name}, maintaining their tone and style.
2. **Be Professional**: Ensure that your response are professional and suitable for a potential client or employer.
3. **Acknowledge Limitations**: If you do not know the answer to a question, clearly state that you do not have the information.
4. **Use Provided Context**: Utilize the summary and LinkedIn profile to inform your responses and provide relevant information.
5. **Engage with Users**: Encourage further questions and maintain an engaging conversation.
6. **Respect User Queries**: Treat all user inquiries with respect and provide thoughtful responses.
7. **JSON Format**: Response must be in JSON format with strict adherence to the provided output schema.
8. **Enclose JSON**: Enclose JSON response with ```json on its own line, and close it with triple backticks on a new line.
9. **Markdown Formatting**: Values for each JSON key **should use** `markdown` formatting, including emphasis, italics, lists etc.

## Output Schema:
${json_schema}

# Additional Notes:
- Ensure that the evaluation reflects the quality of the Agent's response in relation to the provided context.
- Consider the engagement level of the response and its appropriateness for the intended audience.
"""

In [27]:
# Define the evaluator user prompt for assessing the latest response in a conversation:
evaluator_user_prompt = """Here's the conversation between the User and the Agent:
${history}
Here's the latest message from the User:
${message}
Here's the latest response from the Agent:
${reply}
Please evaluate the response, replying with whether it is acceptable and your feedback.
"""

In [28]:
# Define a function to extract and convert structured output from a response text
def structured_output(response_text):
    # Extract the JSON part from the string
    json_part = response_text.split("```json")[1].split("```")[0].strip()
    
    # Convert the extracted string to a JSON object
    json_object = json.loads(json_part)
    
    # Now json_object is a Python dictionary
    return json_object

In [29]:
# Define a function to evaluate the Agent's response
def evaluate(reply, message, history):
    # Construct the messages to be sent to the LLM for evaluation
    messages = [
        {
            "role" : "system",  # Role of the message sender
            "content" : Template(evaluator_system_prompt).substitute(
                name = "Siddharth Singh",  # Substitute the user's name
                linkedin = linkedin,  # Substitute the LinkedIn profile
                summary = summary,  # Substitute the summary
                json_schema = evaluation_json_schema,  # Substitute the JSON schema
            ),
        },
        {
            "role" : "user",  # Role of the user
            "content" : Template(evaluator_user_prompt).substitute(
                history = history,  # Substitute the conversation history
                message = message,  # Substitute th latest user message
                reply = reply  # Substitute the latest agent response
            ),
        },
    ]
    
    # Call the LLM to evaluate the response
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    
    # Return the structured output from the response
    return structured_output(response.choices[0].message.content)

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = Template(system_prompt).substitute(
        name = "Siddharth Singh",  # Substitute the user's name
        linkedin = linkedin,  # Substitute the LinkedIn profile
        summary = summary,  # Substitute the summary
    ) + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt = updated_system_prompt + f"## Your attempted answer : \n{reply}\n\n"
    updated_system_prompt = updated_system_prompt + f"## Reason for rejection : \n{feedback}\n\n"
    
    # Construct the messages to be sent to the LLM
    messages = (
        [
            {
                "role" : "system",  # Role of the message sender
                "content" : updated_system_prompt  # Updated system prompt with previous response, and the feedback from LLM
            }
        ]
        + history  # Include the previous chat history
        + [{"role" : "user", "content" : message}]  # Add the current user message
    )
    
    # Call the Azure OpenAI chat completion API with the constructed messages
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    
    # Return th content of the response from the LLM
    return response.choices[0].message.content

In [33]:
# Function to handle chat interactions with the LLM
def chat(message, history):
    # Construct the messages to be sent to the LLM
    messages = (
        [
            {
                "role" : "system",  # Role of the message sender, indicating this is a system message
                "content" : Template(system_prompt).substitute(
                    name = "Siddharth Singh",  # Substitute the user's name into the system prompt
                    linkedin = linkedin,  # Substitute the LinkedIn profile text into the system prompt
                    summary = summary  # Substitute the summary text into the system prompt
                ),
            }
        ]
        + history  # Include the previous chat history to maintain context
        + [{"role" : "user", "content" : message}]  # Add the current user message to the messages list
    )
    
    done = False  # Initialize a flag to control the loop for processing LLM responses
    while not done:  # Continue processing until the LLM has finished its response
        # Call the LLM with the constructed messages and the tools available for function calls
        response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages, tools = tools)
        
        # Determine the reason for the LLM's response completion
        finish_reason = response.choices[0].finish_reason
        print(f"Finish Reason -> {finish_reason}")  # Log the finish reason for debugging
        
        # If the LLM indicates it wants to call a tool, handle that case
        if finish_reason == "tool_calls":
            # Step 1 - Extract the message from the response, which may contain tool call information
            message = response.choices[0].message
            print(f"Message going inside -> {message}")  # Log the extracted message for debugging
            
            """
            Below is an example of how it may look like -
            Message going inside -> ChatCompletionMessage(content = None, refusal = None, role = "assistant", annotations = [],
            audio = None, function_call = None, tool_calls = [ChatCompletionMessageToolCall(id = "call_mncajkashasklals", 
            function = Function(arguments = '{"question" : "Can you tell me about Nvidia?"}', name = "record_unknown_question"), 
            type = "function")])
            """
            
            # Step 2 - Extract tool_calls details from the 'ChatCompletionMessage'
            tool_calls = message.tool_calls  # Retrieve the list of tool calls requested by the LLM
            print(f"Tool calls -> {tool_calls}")  # Log the tool calls for debugging
            
            """
            Below is an example of how it may look like -
            Message going inside -> [ChatCompletionMessageToolCall(id = "call_mncajkashasklals", 
            function = Function(arguments = '{"question" : "Can you tell me about Nvidia?"}', name = "record_unknown_question"), 
            type = "function")]
            """
            
            # Step 3 - Call the function 'handle_tool_calls' to process the function execution
            results = handle_tool_calls(tool_calls)  # Process the tool calls and obtain results
            print(f"Final Result -> {results}")  # Log the final results for debugging
            
            # Append the LLM's message and the results from tool calls to the message list
            messages.append(message)  # Add the LLM's message to the conversation history
            messages.extend(results)  # Add the results from the tool calls to the conversation history
        else:
            done = True  # Exit the loop if no tool calls are made, indicating the LLM has finished processing
    
    # Return the content of the response from the LLM
    LLM_response = response.choices[0].message.content
    print(f"Reply -> {LLM_response}")
    
    # Evaluate the LLM's response
    evaluation = evaluate(LLM_response, message, history)
    print(f"Evaluation Result -> {evaluation}")
    
    # Extract token usage details
    token_count = response.usage.total_tokens  # Adjust based on the actual structure of the response
    print(f"Token count -> {token_count}")  # Print the token count for monitoring
    
    # Check if the evaluation indicates the response is acceptable
    if evaluation["is_acceptable"]:
        print("Passed Evaluation - Returning reply")
        return LLM_response  # Return the acceptable reply
    else:
        print("Failed Evaluation - Retrying")
        print(f"Feedback received -> {evaluation["feedback"]}")  # Print feedback for debugging
        # new_LLM_response = rerun(LLM_response, message, history, evaluation["feedback"])  # Retry Logic
        # return new_LLM_response  # Return the new reply after retrying
        return LLM_response  # Return the acceptable reply

In [34]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


Finish Reason -> tool_calls
Message going inside -> ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_KYfryzLPEjykFjxyO0mlb9ts', function=Function(arguments='{"question":"Hi, tell me about virat kohli"}', name='record_unknown_question'), type='function')])
Tool calls -> [ChatCompletionMessageToolCall(id='call_KYfryzLPEjykFjxyO0mlb9ts', function=Function(arguments='{"question":"Hi, tell me about virat kohli"}', name='record_unknown_question'), type='function')]
Tool called : record_unknown_question || , arguments passed : {'question': 'Hi, tell me about virat kohli'}
Expected parameters for tool 'record_unknown_question' : ('question',)
Push : Recording Hi, tell me about virat kohli asked that I couldn't answer
Final Result -> [{'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'call_KYfryzLPEjykFjxyO0mlb9ts'}]
Finish Reason -> stop
Reply -> I'm unable to p

## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>